# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo (MC) approach to sample the probable atmospheres that can arise for different planetary conditions.

In [ ]:
from atmodeller import debug_logger
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.constraints import MassConstraint, IronWustiteBufferConstraintHirschmann, SystemConstraints
from atmodeller.interfaces import GasSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2, BasaltLibourelN2
from atmodeller.utilities import earth_oceans_to_kg
from atmodeller.initial_condition import InitialConditionRegressor, InitialConditionSwitchRegressor
import numpy as np
import logging

For production runs, make sure to set the logger to INFO or higher (i.e. WARNING, ERROR, or CRITICAL). Otherwise you will find that your MC runs slower just because of writing the output to the logger.

In [ ]:
logger = debug_logger()
logger.setLevel(logging.INFO)

We now create the species that we are interested in.

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2()))
species

Now create a planet. We recall that we can sample different planetary properties by updating the attributes of this object, even though in this tutorial we don't do this.

In [ ]:
planet: Planet = Planet()

Now set up the main driver of the Monte Carlo (MC) approach. This establishes the ranges over which we sample certain properties.

In [ ]:
def monte_carlo(interior_atmosphere: InteriorAtmosphereSystem, number_of_realisations:int=100):
    """Monte Carlo driver
    
    Args:
        interior_atmosphere: An interior-atmosphere system
        number_of_realisation: Number of simulations to perform
    """

    # Parameters are normally distributed between bounds.
    number_ocean_moles = np.random.uniform(1, 10, number_of_realisations)
    ch_ratios = np.random.uniform(0.1, 1, number_of_realisations)
    fo2_shifts = np.random.uniform(-4, 4, number_of_realisations)

    # ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
    N_ppmw = 2.8

    # The nitrogen mass is constant
    mass_N = N_ppmw * 1.0e-6 * planet.mantle_mass

    for realisation in range(number_of_realisations):

        mass_H = earth_oceans_to_kg(number_ocean_moles[realisation])
        mass_C = ch_ratios[realisation] * mass_H
        constraints = SystemConstraints([
            MassConstraint(species="H", value=mass_H),
            MassConstraint(species="C", value=mass_C),
            MassConstraint(species="N", value=mass_N),
            IronWustiteBufferConstraintHirschmann(log10_shift=fo2_shifts[realisation])
        ])
        interior_atmosphere.solve(constraints, factor=0.1)


We can run the MC as follows. This may take a minute or two to run.

In [ ]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)
monte_carlo(interior_atmosphere)

The simulation data can be exported to an Excel or a pickle file by setting the appropriate keyword argument in the output method:

In [ ]:
interior_atmosphere.output(file_prefix='tutorial3_monte_carlo', to_excel=True, to_pickle=True)

If you just want to access a dictionary of the data, which contains all the dataframes, you can call the output method with no arguments:

In [ ]:
output_data = interior_atmosphere.output()

When performing a MC simulation, sometimes problems can arise when chosen model parameters would result in a solution that is far from the initial guess. Internally, atmodeller chooses an initial guess for the solution and uses this as a starting point for the numerical solution technique. But if this initial guess is far from the actual solution, the solver may fail. To address this, it is often convenient to run a smaller MC simulation with reduced parameter bounds in order to generate some output. Then we can use this output to train a new initial condition to provide an improved initial guess for a new MC run.

In the following, we use the generated output from the previous run to inform the selection of the initial condition:

In [ ]:
initial_condition = InitialConditionRegressor('tutorial3_monte_carlo.pkl', fit=True, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

In the above, fit = True, which means the trained data from the previous run (as computed from the output in the pickle file) is only used for the first fit_batch_size = 100 simulations. Subsequently the regressor will re-train itself on just the (fit_batch_size = 100) samples generated from the current model, discarding knowledge of the previous data it was trained on. Then, every partial_fit_batch_size = 100 simulations, it will update its training with the last batch of newly generated samples in order to better inform the selection of subsequent initial conditions. This is known as a dynamic or online learning approach.

It is necessary to pass the initial condition to the interior atmosphere system when it is created:

In [ ]:
interior_atmosphere_ic: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, initial_condition=initial_condition, planet=planet)
monte_carlo(interior_atmosphere_ic, number_of_realisations=200)

If you compare the log output for the two MC runs, you will see in the second MC example that the initial condition re-trained itself after 100 samples had been generated and then partially retrained itself every 50 samples. This keeps the RMSE between the initial guess and actual solution to a smaller value than simply guessing a constant initial condition. Also, fit = True allows you to train an initial condition on a similar but not identical model (for example, different solubility laws or gas equations of states), where once enough samples have been generated you would prefer to only use the new model to generate new estimates (since the behavior of the new model and the previous similar-but-not-the-same model will diverge).

If you want to combine an initial condition that is constant, and then switches to a regressor, you can use:

In [ ]:
initial_condition_switch = InitialConditionSwitchRegressor(value=10, fit=True, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

You must specify the initial constant value first, and then the arguments associated with the regressor. In the above, a constant initial condition will be used for the first 100 samples, after which the initial condition will train itself on those first 100 samples. Then, for every subsequent 50 samples, the initial condition will partially re-train itself. This approach can work well, but of course it relies on finding those first 100 samples to train the regressor.

Although it is tempting to set fit_batch_size to as small a number as possible to begin training the initial condition, formally this initial sample should capture some of the variability in the solution since it is used to calibrate the scalings. This is important because the scalings are fixed for an initial condition regressor and are not updated (unlike the trained model) during partial fitting. Hence in practice, it may be necessary to incrementally add complexity in terms of solubility and equations of state, where the initial condition for each MC is trained on a previous simpler MC run.

In [ ]:
interior_atmosphere_ic_switch: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, initial_condition=initial_condition_switch, planet=planet)
monte_carlo(interior_atmosphere_ic_switch, number_of_realisations=200)